# Bonus 1 y Bonus 2

Bonus 1

El Bonus 1 lo hago en el Bonus 2, ya que al automatizarlo me es más sencillo cambiar parámetros e ir comprobando resultados.

Y más teniendo en cuenta que a la que le meta un poco de caña al ordenador me peta.

Bonus 2

In [28]:
def Pipeline(s, n_samples, n_words):
    import re
    import nltk
    from nltk import sent_tokenize, word_tokenize
    from nltk.stem import WordNetLemmatizer
    from nltk.stem import SnowballStemmer
    from nltk import FreqDist
    from nltk.tokenize import word_tokenize
    from nltk.sentiment.vader import SentimentIntensityAnalyzer
    from stop_words import get_stop_words
    
    def clean_up(s):
        s = re.sub(r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', '', s)
        s = re.sub('[^a-zA-Z]+', ' ', s)
        s = s.lower()
        return s

    def tokenize(s):
        return word_tokenize(s)

    def stem_and_lemmatize(l):
        return [WordNetLemmatizer().lemmatize(nltk.stem.SnowballStemmer('english').stem(i)) for i in l]

    def remove_stopwords(l):
        return [word for word in l if word not in get_stop_words('en')]
    
    sample = tweer_data.sample(n=n_samples, random_state=1)
             
    sample['text_processed'] = sample['text'].apply(clean_up)
    sample['text_processed'] = sample['text_processed'].apply(tokenize).apply(stem_and_lemmatize).apply(remove_stopwords)
             
    words = []
    for x in sample.text_processed:
        words += x
             
    top_words = sorted(FreqDist(words), key=FreqDist(words).get, reverse=True)[:n_words]
             
    def find_features(document):
        words = set(document)
        features = {}
        for w in top_words:
            features[w] = (w in words)
        sentiment = SentimentIntensityAnalyzer().polarity_scores(" ".join(document))
        if sentiment["pos"] > 0.3:
            sentiment = True
        else:
            sentiment = False
        return (features, sentiment)
             
    features_in_text = sample['text_processed'].apply(find_features)
             
    train_set, test_set = features_in_text[:int(n_samples*0.8)], features_in_text[int(n_samples*0.8):]
    classifier = nltk.NaiveBayesClassifier.train(train_set)
    
    return "Classifier accuracy percent:",(nltk.classify.accuracy(classifier, test_set))*100

In [31]:
import pandas as pd

tweer_data = pd.read_csv('./Sentiment140.csv')

print(Pipeline(tweer_data.text, 5000, 7000))

('Classifier accuracy percent:', 79.5)
